In [ ]:
# ============================================================================
# SUPERNUTRI-SCORE - Notebook Jupyter
# Système d'évaluation transparent des aliments
# Nutri-Score + ELECTRE TRI + SuperScore
# ============================================================================

# %% [markdown]
# # SuperNutri-Score - Analyse Complète
# 
# 1. **Calcul du Nutri-Score** selon la méthodologie officielle (Mars 2025)
# 2. **Méthode ELECTRE TRI** pour la classification multicritère
# 3. **Comparaison** avec les scores OpenFoodFacts
# 4. **Visualisations** des résultats

# %% [markdown]
# ## Installation et Import des Bibliothèques

# %%
# !pip install pandas numpy scikit-learn matplotlib seaborn openpyxl

# %%
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


# %% [markdown]
# ## Classe DataLoader - Chargement des Données

# %%
class DataLoader:
    """
    Classe pour charger et préparer les données depuis Excel
    """
    
    @staticmethod
    def charger_excel(chemin_fichier: str) -> pd.DataFrame:
        """
        Charge le fichier Excel et standardise les noms de colonnes
        
        Args:
            chemin_fichier: Chemin vers le fichier Excel
            
        Returns:
            DataFrame avec colonnes standardisées
        """
        print(f"Chargement du fichier: {chemin_fichier}")
        df = pd.read_excel(chemin_fichier)
        
        print(f"{len(df)} produits chargés")
        print(f"Colonnes trouvées: {list(df.columns)}\n")
        
        # Mapping des colonnes vers les noms standardisés
        colonnes_mapping = {
            'Nom du produit': 'nom',
            'Énergie (kJ ou kcal / 100g)': 'energie_kj', 
            'Acides gras saturés (g / 100g)': 'graisses_saturees',
            'Sucres (g / 100g)': 'sucres',
            'Sodium (mg / 100g)': 'sodium',  
            'Sel (g / 100g)': 'sel',
            'Protéines (g / 100g)': 'proteines',
            'Fibres (g / 100g)': 'fibres',
            'Fruits/légumes/noix (%)': 'fruits_legumes_noix',
            'Nombre d\'additifs': 'additifs_n',
            "Nombre d'additifs": 'additifs_n',
            'Score Nutri-score (valeur numérique)': 'nutriscore_score',
            'Label Nutri-score (A, B, C, D, E)': 'nutriscore_grade',
            'Label Green-score (A, B, C, D, E)': 'ecoscore_grade',
            'Score Green-score': 'ecoscore_score',
            'Label bio (oui/non)': 'bio'
        }
        
        # Renommer les colonnes
        df_clean = df.rename(columns=colonnes_mapping)
        
        # Conversion du label bio
        if 'bio' in df_clean.columns:
            df_clean['bio'] = df_clean['bio'].apply(
                lambda x: 1 if str(x).lower() in ['oui', 'yes', '1', 'true'] else 0
            )
        
        # Nettoyer les données manquantes
        df_clean = DataLoader._nettoyer_donnees(df_clean)
        
        return df_clean
    





    #il faut l'enlever
    @staticmethod
    def _nettoyer_donnees(df: pd.DataFrame) -> pd.DataFrame:
        """Nettoie les données manquantes"""
        colonnes_numeriques = [
            'energie_kj', 'graisses_saturees', 'sucres', 'sodium',
            'proteines', 'fibres', 'fruits_legumes_noix', 'additifs_n'
        ]
        
        for col in colonnes_numeriques:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            else:
                if col == 'additifs_n':
                    print(f"Colonne '{col}' non trouvée, création avec valeurs à 0")
                    df[col] = 0
        
        if 'nutriscore_grade' in df.columns:
            df['nutriscore_grade'] = df['nutriscore_grade'].astype(str).str.strip().str.upper()
            df['nutriscore_grade'] = df['nutriscore_grade'].replace({'NAN': None, '': None})
        
        if 'ecoscore_grade' in df.columns:
            df['ecoscore_grade'] = df['ecoscore_grade'].astype(str).str.strip().str.upper()
            df['ecoscore_grade'] = df['ecoscore_grade'].replace({'NAN': None, '': None})
        
        print(f"Données nettoyées et standardisées")
        print(f"   • Énergie: Min={df['energie_kj'].min():.0f} kJ, Max={df['energie_kj'].max():.0f} kJ")
        print(f"   • Sodium: Min={df['sodium'].min():.0f} mg, Max={df['sodium'].max():.0f} mg")
        print()
        
        return df

# %% [markdown]
# ## Classe NutriScoreCalculator

# %%
class NutriScoreCalculator:
    """
    Calculateur du Nutri-Score selon la méthodologie officielle
    """
    
    # Table Énergie (kJ/100g) - Image 1
    # 0 pt si ≤335, 1 pt si >335, 2 pt si >670, etc.
    POINTS_ENERGIE = [
        (0, 0),       # Départ
        (335, 1),     # >335 → 1 pt
        (670, 2),     # >670 → 2 pts
        (1005, 3),    # >1005 → 3 pts
        (1340, 4),    # >1340 → 4 pts
        (1675, 5),    # >1675 → 5 pts
        (2010, 6),    # >2010 → 6 pts
        (2345, 7),    # >2345 → 7 pts
        (2680, 8),    # >2680 → 8 pts
        (3015, 9),    # >3015 → 9 pts
        (3350, 10)    # >3350 → 10 pts
    ]
    
    # Table Sucres (g/100g) - Image 1
    # 0 pt si ≤3.4, 1 pt si >3.4, etc.
    POINTS_SUCRES = [
        (0, 0),       # Départ
        (3.4, 1),     # >3.4 → 1 pt
        (6.8, 2),     # >6.8 → 2 pts
        (10, 3),      # >10 → 3 pts
        (14, 4),      # >14 → 4 pts
        (17, 5),      # >17 → 5 pts
        (20, 6),      # >20 → 6 pts
        (24, 7),      # >24 → 7 pts
        (27, 8),      # >27 → 8 pts
        (31, 9),      # >31 → 9 pts
        (34, 10),     # >34 → 10 pts
        (37, 11),     # >37 → 11 pts
        (41, 12),     # >41 → 12 pts
        (44, 13),     # >44 → 13 pts
        (48, 14),     # >48 → 14 pts
        (51, 15)      # >51 → 15 pts
    ]
    
    # Table Acides Gras Saturés (g/100g) - Image 1
    # 0 pt si ≤1, 1 pt si >1, etc.
    POINTS_AG_SATURES = [
        (0, 0),       # Départ
        (1, 1),       # >1 → 1 pt
        (2, 2),       # >2 → 2 pts
        (3, 3),       # >3 → 3 pts
        (4, 4),       # >4 → 4 pts
        (5, 5),       # >5 → 5 pts
        (6, 6),       # >6 → 6 pts
        (7, 7),       # >7 → 7 pts
        (8, 8),       # >8 → 8 pts
        (9, 9),       # >9 → 9 pts
        (10, 10)      # >10 → 10 pts
    ]
    
    # Table Sel (g/100g) - Image 1
    # Conversion en Sodium (mg): Sel (g) × 400 = Sodium (mg)
    # 0 pt si ≤0.2g sel (80mg sodium), 1 pt si >0.2g sel (>80mg), etc.
    POINTS_SODIUM = [
        (0, 0),       # Départ
        (80, 1),      # >0.2g sel (>80mg sodium) → 1 pt
        (160, 2),     # >0.4g → 2 pts
        (240, 3),     # >0.6g → 3 pts
        (320, 4),     # >0.8g → 4 pts
        (400, 5),     # >1.0g → 5 pts
        (480, 6),     # >1.2g → 6 pts
        (560, 7),     # >1.4g → 7 pts
        (640, 8),     # >1.6g → 8 pts
        (720, 9),     # >1.8g → 9 pts
        (800, 10),    # >2.0g → 10 pts
        (880, 11),    # >2.2g → 11 pts
        (960, 12),    # >2.4g → 12 pts
        (1040, 13),   # >2.6g → 13 pts
        (1120, 14),   # >2.8g → 14 pts
        (1200, 15),   # >3.0g → 15 pts
        (1280, 16),   # >3.2g → 16 pts
        (1360, 17),   # >3.4g → 17 pts
        (1440, 18),   # >3.6g → 18 pts
        (1520, 19),   # >3.8g → 19 pts
        (1600, 20)    # >4.0g → 20 pts
    ]
    
    # Table Fibres (g/100g) - Image 2
    # 0 pt si ≤3.0, 1 pt si >3.0, etc.
    POINTS_FIBRES = [
        (0, 0),       # Départ
        (3.0, 1),     # >3.0 → 1 pt
        (4.1, 2),     # >4.1 → 2 pts
        (5.2, 3),     # >5.2 → 3 pts
        (6.3, 4),     # >6.3 → 4 pts
        (7.4, 5)      # >7.4 → 5 pts
    ]
    
    # Table Protéines (g/100g) - Image 2
    # 0 pt si ≤2.4, 1 pt si >2.4, etc.
    POINTS_PROTEINES = [
        (0, 0),       # Départ
        (2.4, 1),     # >2.4 → 1 pt
        (4.8, 2),     # >4.8 → 2 pts
        (7.2, 3),     # >7.2 → 3 pts
        (9.6, 4),     # >9.6 → 4 pts
        (12, 5),      # >12 → 5 pts
        (14, 6),      # >14 → 6 pts
        (17, 7)       # >17 → 7 pts
    ]
    
    # Table Fruits/Légumes/Légumineuses/Noix (%) - Image 2
    # 0 pt si ≤40, 1 pt si >40, 2 pt si >60, 5 pt si >80
    POINTS_FRUITS_LEGUMES = [
        (0, 0),       # Départ
        (40, 1),      # >40 → 1 pt
        (60, 2),      # >60 → 2 pts
        (80, 5)       # >80 → 5 pts
    ]
    
    # Seuils des catégories - Image 4
    CATEGORIES = [
        (0, 'A'),      # Min à 0
        (2, 'B'),      # 1 à 2
        (10, 'C'),     # 3 à 10
        (18, 'D'),     # 11 à 18
        (float('inf'), 'E')  # 19 à max
    ]
    
    @staticmethod
    def _get_points_negatifs(valeur: float, table: List[Tuple[float, int]]) -> int:
        """
        Pour les critères NÉGATIFS (énergie, sucres, AG saturés, sodium)
        Plus la valeur est élevée, plus on a de points négatifs
        Logique: Si valeur > seuil, alors on prend les points correspondants
        """
        points = 0
        for seuil, pts in table:
            if valeur > seuil:
                points = pts
            else:
                break
        return points
    
    @staticmethod
    def _get_points_positifs(valeur: float, table: List[Tuple[float, int]]) -> int:
        """
        Pour les critères POSITIFS (fibres, protéines, fruits/légumes)
        Plus la valeur est élevée, plus on a de points positifs
        Logique: Si valeur > seuil, alors on prend les points correspondants
        """
        points = 0
        for seuil, pts in table:
            if valeur > seuil:
                points = pts
            else:
                break
        return points
    
    def calculer_score(self, energie_kj: float, sucres_g: float, ag_satures_g: float,
                      sodium_mg: float, proteines_g: float, fibres_g: float, 
                      fruits_legumes_pct: float) -> Dict:
        """Calcule le Nutri-Score d'un aliment"""
        
        # Composante NÉGATIVE (à limiter) - logique: valeur > seuil
        points_energie = self._get_points_negatifs(energie_kj, self.POINTS_ENERGIE)
        points_sucres = self._get_points_negatifs(sucres_g, self.POINTS_SUCRES)
        points_ag_satures = self._get_points_negatifs(ag_satures_g, self.POINTS_AG_SATURES)
        points_sodium = self._get_points_negatifs(sodium_mg, self.POINTS_SODIUM)
        
        composante_negative = points_energie + points_sucres + points_ag_satures + points_sodium
        
        # Composante POSITIVE (à favoriser) - logique: valeur > seuil
        points_fibres = self._get_points_positifs(fibres_g, self.POINTS_FIBRES)
        points_proteines = self._get_points_positifs(proteines_g, self.POINTS_PROTEINES)
        points_fruits_legumes = self._get_points_positifs(fruits_legumes_pct, self.POINTS_FRUITS_LEGUMES)
        
        # Règle spéciale des protéines
        if composante_negative >= 11 and fruits_legumes_pct < 80:
            composante_positive = points_fibres + points_fruits_legumes
            proteines_comptees = False
        else:
            composante_positive = points_fibres + points_proteines + points_fruits_legumes
            proteines_comptees = True
        
        score_final = composante_negative - composante_positive
        
        categorie = 'E'
        for seuil, cat in self.CATEGORIES:
            if score_final <= seuil:
                categorie = cat
                break
        
        return {
            'score': score_final,
            'categorie': categorie,
            'composante_negative': composante_negative,
            'composante_positive': composante_positive,
            'proteines_comptees': proteines_comptees
        }
    
    def calculer_pour_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calcule le Nutri-Score pour tous les produits"""
        resultats = []
        
        for idx, row in df.iterrows():
            try:
                resultat = self.calculer_score(
                    energie_kj=row['energie_kj'],
                    sucres_g=row['sucres'],
                    ag_satures_g=row['graisses_saturees'],
                    sodium_mg=row['sodium'],
                    proteines_g=row['proteines'],
                    fibres_g=row['fibres'],
                    fruits_legumes_pct=row['fruits_legumes_noix']
                )
                resultats.append(resultat)
            except Exception as e:
                print(f"Erreur ligne {idx}: {e}")
                resultats.append({
                    'score': None,
                    'categorie': None,
                    'composante_negative': None,
                    'composante_positive': None
                })
        
        df_resultats = df.copy()
        df_resultats['nutriscore_calcule'] = [r['categorie'] for r in resultats]
        df_resultats['score_calcule'] = [r['score'] for r in resultats]
        df_resultats['composante_neg'] = [r['composante_negative'] for r in resultats]
        df_resultats['composante_pos'] = [r['composante_positive'] for r in resultats]
        
        return df_resultats

# %% [markdown]
# ## Classe ElectreTRI

# %%
class ElectreTRI:
    """Implémentation de la méthode ELECTRE TRI"""
    
    def __init__(self, profils: pd.DataFrame, poids: Dict[str, float], 
                 seuil_majorite: float = 0.6, criteres_a_minimiser: List[str] = None):
        self.profils = profils
        self.poids = poids
        self.seuil_majorite = seuil_majorite
        self.criteres_a_minimiser = criteres_a_minimiser or ['en', 'sa', 'su', 'so', 'ad']
        
        somme_poids = sum(poids.values())
        self.poids_normalises = {k: v/somme_poids for k, v in poids.items()}
    
    def _concordance_partielle(self, valeur_aliment: float, valeur_profil: float, 
                               critere: str, sens: str) -> int:
        a_minimiser = critere in self.criteres_a_minimiser
        
        if sens == 'aliment_profil':
            if a_minimiser:
                return 1 if valeur_profil >= valeur_aliment else 0
            else:
                return 1 if valeur_aliment >= valeur_profil else 0
        else:
            if a_minimiser:
                return 1 if valeur_aliment >= valeur_profil else 0
            else:
                return 1 if valeur_profil >= valeur_aliment else 0
    
    def _concordance_globale(self, aliment: pd.Series, profil: pd.Series, sens: str) -> float:
        concordance = 0
        for critere, poids in self.poids_normalises.items():
            c_partiel = self._concordance_partielle(
                aliment[critere], profil[critere], critere, sens
            )
            concordance += poids * c_partiel
        return concordance
    
    def _surclasse(self, aliment: pd.Series, profil: pd.Series) -> Tuple[bool, bool]:
        c_aliment_profil = self._concordance_globale(aliment, profil, 'aliment_profil')
        c_profil_aliment = self._concordance_globale(aliment, profil, 'profil_aliment')
        
        aliment_S_profil = c_aliment_profil >= self.seuil_majorite
        profil_S_aliment = c_profil_aliment >= self.seuil_majorite
        
        return aliment_S_profil, profil_S_aliment
    
    def affectation_pessimiste(self, aliment: pd.Series) -> str:
        categories = ['E\'', 'D\'', 'C\'', 'B\'', 'A\'']
        profils_ordre = ['b6', 'b5', 'b4', 'b3', 'b2', 'b1']
        
        for i, profil_nom in enumerate(profils_ordre):
            if profil_nom not in self.profils.index:
                continue
            
            profil = self.profils.loc[profil_nom]
            aliment_S_profil, _ = self._surclasse(aliment, profil)
            
            if aliment_S_profil:
                if i < len(categories):
                    return categories[i]
        
        return 'E\''
    
    def affectation_optimiste(self, aliment: pd.Series) -> str:
        categories = ['A\'', 'B\'', 'C\'', 'D\'', 'E\'']
        profils_ordre = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6']
        
        for i, profil_nom in enumerate(profils_ordre):
            if profil_nom not in self.profils.index:
                continue
            
            profil = self.profils.loc[profil_nom]
            aliment_S_profil, profil_S_aliment = self._surclasse(aliment, profil)
            
            if profil_S_aliment and not aliment_S_profil:
                if i > 0:
                    return categories[5 - i]
        
        return 'A\''
    
    def classifier_base_donnees(self, df: pd.DataFrame) -> pd.DataFrame:
        """Classifie tous les aliments"""
        resultats = df.copy()
        
        resultats['Categorie_Pessimiste'] = resultats.apply(
            lambda row: self.affectation_pessimiste(row), axis=1
        )
        
        resultats['Categorie_Optimiste'] = resultats.apply(
            lambda row: self.affectation_optimiste(row), axis=1
        )
        
        return resultats

# %% [markdown]
# ## Classe ComparateurResultats

# %%
class ComparateurResultats:
    """Classe pour comparer les résultats et générer des métriques"""
    
    @staticmethod
    def matrice_confusion(y_true: List, y_pred: List, labels: List[str] = None, 
                         titre: str = "Matrice de Confusion") -> plt.Figure:
        """Génère une matrice de confusion"""
        if labels is None:
            labels = ['A', 'B', 'C', 'D', 'E']
        
        cm = confusion_matrix(y_true, y_pred, labels=labels)
        
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=labels, yticklabels=labels, ax=ax, cbar_kws={'label': 'Nombre de produits'})
        ax.set_xlabel('Prédit (Notre calcul)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Réel (OpenFoodFacts)', fontsize=12, fontweight='bold')
        ax.set_title(titre, fontsize=14, fontweight='bold', pad=20)
        
        plt.tight_layout()
        return fig
    
    @staticmethod
    def rapport_classification(y_true: List, y_pred: List, labels: List[str] = None) -> str:
        """Génère un rapport de classification détaillé"""
        if labels is None:
            labels = ['A', 'B', 'C', 'D', 'E']
        
        rapport = classification_report(y_true, y_pred, labels=labels, 
                                       target_names=labels, zero_division=0)
        
        accuracy = accuracy_score(y_true, y_pred)
        
        return f"Accuracy (Précision globale): {accuracy:.2%}\n\n{rapport}"
    
    @staticmethod
    def distribution_categories(df: pd.DataFrame, col_reel: str, col_predit: str) -> plt.Figure:
        """Compare la distribution des catégories"""
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        counts_reel = df[col_reel].value_counts().reindex(['A', 'B', 'C', 'D', 'E'], fill_value=0)
        counts_reel.plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
        axes[0].set_title('Distribution Nutri-Score Réel\n(OpenFoodFacts)', 
                         fontsize=12, fontweight='bold')
        axes[0].set_xlabel('Catégorie', fontweight='bold')
        axes[0].set_ylabel('Nombre de produits', fontweight='bold')
        axes[0].tick_params(axis='x', rotation=0)
        axes[0].grid(axis='y', alpha=0.3)
        
        counts_pred = df[col_predit].value_counts().reindex(['A', 'B', 'C', 'D', 'E'], fill_value=0)
        counts_pred.plot(kind='bar', ax=axes[1], color='coral', edgecolor='black')
        axes[1].set_title('Distribution Nutri-Score Calculé\n(Notre algorithme)', 
                         fontsize=12, fontweight='bold')
        axes[1].set_xlabel('Catégorie', fontweight='bold')
        axes[1].set_ylabel('Nombre de produits', fontweight='bold')
        axes[1].tick_params(axis='x', rotation=0)
        axes[1].grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        return fig
    
    @staticmethod
    def analyse_concordance(df: pd.DataFrame, col_reel: str, col_predit: str) -> Dict:
        """Analyse la concordance entre les classifications"""
        df_clean = df.dropna(subset=[col_reel, col_predit])
        
        concordance_exacte = (df_clean[col_reel] == df_clean[col_predit]).sum()
        total = len(df_clean)
        taux_concordance = concordance_exacte / total if total > 0 else 0
        
        categories_ordre = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
        
        diff = df_clean.apply(
            lambda row: abs(categories_ordre.get(row[col_reel], 0) - 
                          categories_ordre.get(row[col_predit], 0)), axis=1
        )
        
        concordance_1_cat = (diff <= 1).sum()
        taux_concordance_1 = concordance_1_cat / total if total > 0 else 0
        
        return {
            'total_produits': total,
            'concordance_exacte': concordance_exacte,
            'taux_concordance_exacte': taux_concordance,
            'concordance_1_categorie': concordance_1_cat,
            'taux_concordance_1_categorie': taux_concordance_1,
            'differences': diff.value_counts().to_dict()
        }

# %% [markdown]
# ---
# # ANALYSE PRINCIPALE
# ---

# %% [markdown]
# ## Chargement des Données

# %%
CHEMIN_FICHIER = "DB_SAUCES_CLEAN.xlsx" 

# Chargement
loader = DataLoader()
df = loader.charger_excel(CHEMIN_FICHIER)

# Aperçu des données
print("Aperçu des premières lignes:")
df[['nom', 'energie_kj', 'sucres', 'sodium', 'nutriscore_grade']].head(10)

# %% [markdown]
# ## Calcul du Nutri-Score

# %%
print("Calcul du Nutri-Score pour tous les produits...")
calc = NutriScoreCalculator()
df_resultats = calc.calculer_pour_dataframe(df)

print("Calcul terminé!\n")

# Afficher quelques résultats
print("Exemples de résultats:")
df_resultats[['nom', 'nutriscore_grade', 'nutriscore_calcule', 'score_calcule']].head(10)

# %% [markdown]
# ## Analyse Comparative avec OpenFoodFacts

# %%
comparateur = ComparateurResultats()

if 'nutriscore_grade' in df_resultats.columns:
    # Analyse de concordance
    analyse = comparateur.analyse_concordance(
        df_resultats, 'nutriscore_grade', 'nutriscore_calcule'
    )
    
    print("=" * 80)
    print("ANALYSE DE CONCORDANCE")
    print("=" * 80)
    print(f"Total de produits analysés: {analyse['total_produits']}")
    print(f"Concordance exacte: {analyse['concordance_exacte']} produits ({analyse['taux_concordance_exacte']:.1%})")
    print(f"Concordance à ±1 catégorie: {analyse['concordance_1_categorie']} produits ({analyse['taux_concordance_1_categorie']:.1%})")
    print(f"\nRépartition des différences:")
    for diff, count in sorted(analyse['differences'].items()):
        print(f"  Différence de {diff} catégorie(s): {count} produits")

# %%
# Matrice de confusion
if 'nutriscore_grade' in df_resultats.columns:
    y_true = df_resultats['nutriscore_grade'].dropna()
    y_pred = df_resultats.loc[y_true.index, 'nutriscore_calcule']
    
    fig = comparateur.matrice_confusion(
        y_true, y_pred,
        titre="Matrice de Confusion: Nutri-Score Réel vs Calculé"
    )
    plt.show()

# %%
# Distribution des catégories
if 'nutriscore_grade' in df_resultats.columns:
    fig = comparateur.distribution_categories(
        df_resultats, 'nutriscore_grade', 'nutriscore_calcule'
    )
    plt.show()

# %%
# Rapport de classification détaillé
if 'nutriscore_grade' in df_resultats.columns:
    print("=" * 80)
    print("RAPPORT DE CLASSIFICATION DÉTAILLÉ")
    print("=" * 80)
    print(comparateur.rapport_classification(y_true, y_pred))

# %% [markdown]
# ## Application d'ELECTRE TRI

# %%
# Préparation des données pour ELECTRE TRI
df_electre = df_resultats.copy()
df_electre['en'] = df_electre['energie_kj']
df_electre['sa'] = df_electre['graisses_saturees']
df_electre['su'] = df_electre['sucres']
df_electre['so'] = df_electre['sodium']
df_electre['pr'] = df_electre['proteines']
df_electre['fi'] = df_electre['fibres']
df_electre['fr'] = df_electre['fruits_legumes_noix']

# Gestion de la colonne additifs (peut avoir différents noms)
if 'additifs_n' in df_electre.columns:
    df_electre['ad'] = df_electre['additifs_n']
elif 'Nombre d\'additifs' in df_electre.columns:
    df_electre['ad'] = df_electre['Nombre d\'additifs']
else:
    print("Colonne additifs non trouvée, utilisation de 0 par défaut")
    df_electre['ad'] = 0

# Définition des profils limites
profils_data = {
    'en': [0, 500, 800, 1200, 1800, 4000],      # Énergie (kJ)
    'sa': [0, 0.5, 1.5, 3.0, 5.0, 20],          # AG saturés (g)
    'su': [0, 3, 6, 10, 15, 50],                # Sucres (g)
    'so': [0, 200, 400, 600, 900, 2000],        # Sodium (mg)
    'pr': [10, 3, 2, 1, 0.5, 0],                # Protéines (g)
    'fi': [10, 2.5, 1.5, 0.8, 0.3, 0],          # Fibres (g)
    'fr': [100, 80, 60, 40, 20, 0],             # Fruits/légumes (%)
    'ad': [0, 3, 6, 10, 15, 30]                 # Additifs
}

profils = pd.DataFrame(profils_data, index=['b6', 'b5', 'b4', 'b3', 'b2', 'b1'])

print("Profils limites définis:")
profils

# %%
# Définition des poids
poids = {
    'en': 0.15,  # Énergie
    'sa': 0.15,  # AG saturés
    'su': 0.15,  # Sucres
    'so': 0.20,  # Sodium (important pour les sauces)
    'pr': 0.10,  # Protéines
    'fi': 0.10,  # Fibres
    'fr': 0.10,  # Fruits/légumes
    'ad': 0.05   # Additifs
}

print("Poids des critères:")
for critere, poids_val in poids.items():
    print(f"  {critere}: {poids_val*100:.0f}%")

# %% [markdown]
# ## Classification ELECTRE TRI avec λ = 0.6

# %%
print("=" * 80)
print("CLASSIFICATION ELECTRE TRI (λ = 0.6)")
print("=" * 80)

electre_06 = ElectreTRI(profils, poids, seuil_majorite=0.6)
df_electre_06 = electre_06.classifier_base_donnees(df_electre)

print("\nClassification terminée avec λ = 0.6\n")

# Afficher les résultats
print("Exemples de classifications:")
colonnes_affichage = ['nom', 'nutriscore_grade', 'nutriscore_calcule', 
                     'Categorie_Pessimiste', 'Categorie_Optimiste']
df_electre_06[colonnes_affichage].head(15)

# %%
# Distribution des catégories ELECTRE TRI (λ = 0.6)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pessimiste
counts_pess = df_electre_06['Categorie_Pessimiste'].value_counts().reindex(
    ['A\'', 'B\'', 'C\'', 'D\'', 'E\''], fill_value=0
)
counts_pess.plot(kind='bar', ax=axes[0], color='#2ecc71', edgecolor='black')
axes[0].set_title('ELECTRE TRI - Affectation Pessimiste\n(λ = 0.6)', 
                 fontsize=12, fontweight='bold')
axes[0].set_xlabel('Catégorie', fontweight='bold')
axes[0].set_ylabel('Nombre de produits', fontweight='bold')
axes[0].tick_params(axis='x', rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Optimiste
counts_opt = df_electre_06['Categorie_Optimiste'].value_counts().reindex(
    ['A\'', 'B\'', 'C\'', 'D\'', 'E\''], fill_value=0
)
counts_opt.plot(kind='bar', ax=axes[1], color='#e74c3c', edgecolor='black')
axes[1].set_title('ELECTRE TRI - Affectation Optimiste\n(λ = 0.6)', 
                 fontsize=12, fontweight='bold')
axes[1].set_xlabel('Catégorie', fontweight='bold')
axes[1].set_ylabel('Nombre de produits', fontweight='bold')
axes[1].tick_params(axis='x', rotation=0)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## Classification ELECTRE TRI avec λ = 0.7

# %%
print("=" * 80)
print("CLASSIFICATION ELECTRE TRI (λ = 0.7)")
print("=" * 80)

electre_07 = ElectreTRI(profils, poids, seuil_majorite=0.7)
df_electre_07 = electre_07.classifier_base_donnees(df_electre)

print("\nClassification terminée avec λ = 0.7\n")

# Afficher les résultats
print("Exemples de classifications:")
df_electre_07[colonnes_affichage].head(15)

# %%
# Comparaison des deux seuils
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# λ = 0.6 Pessimiste
counts_06_pess = df_electre_06['Categorie_Pessimiste'].value_counts().reindex(
    ['A\'', 'B\'', 'C\'', 'D\'', 'E\''], fill_value=0
)
counts_06_pess.plot(kind='bar', ax=axes[0, 0], color='#3498db', edgecolor='black')
axes[0, 0].set_title('ELECTRE TRI Pessimiste (λ = 0.6)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Nombre de produits', fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=0)
axes[0, 0].grid(axis='y', alpha=0.3)

# λ = 0.6 Optimiste
counts_06_opt = df_electre_06['Categorie_Optimiste'].value_counts().reindex(
    ['A\'', 'B\'', 'C\'', 'D\'', 'E\''], fill_value=0
)
counts_06_opt.plot(kind='bar', ax=axes[0, 1], color='#9b59b6', edgecolor='black')
axes[0, 1].set_title('ELECTRE TRI Optimiste (λ = 0.6)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Nombre de produits', fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=0)
axes[0, 1].grid(axis='y', alpha=0.3)

# λ = 0.7 Pessimiste
counts_07_pess = df_electre_07['Categorie_Pessimiste'].value_counts().reindex(
    ['A\'', 'B\'', 'C\'', 'D\'', 'E\''], fill_value=0
)
counts_07_pess.plot(kind='bar', ax=axes[1, 0], color='#e67e22', edgecolor='black')
axes[1, 0].set_title('ELECTRE TRI Pessimiste (λ = 0.7)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Catégorie', fontweight='bold')
axes[1, 0].set_ylabel('Nombre de produits', fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=0)
axes[1, 0].grid(axis='y', alpha=0.3)

# λ = 0.7 Optimiste
counts_07_opt = df_electre_07['Categorie_Optimiste'].value_counts().reindex(
    ['A\'', 'B\'', 'C\'', 'D\'', 'E\''], fill_value=0
)
counts_07_opt.plot(kind='bar', ax=axes[1, 1], color='#1abc9c', edgecolor='black')
axes[1, 1].set_title('ELECTRE TRI Optimiste (λ = 0.7)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Catégorie', fontweight='bold')
axes[1, 1].set_ylabel('Nombre de produits', fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=0)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## SuperNutri-Score - Modèle Combiné

# %%
def calculer_supernutriscore(row):
    """
    Calcule le SuperNutri-Score en combinant:
    - ELECTRE TRI (pessimiste)
    - Eco-Score
    - Label Bio
    """
    # Conversion des catégories en scores numériques
    scores_electre = {'A\'': 5, 'B\'': 4, 'C\'': 3, 'D\'': 2, 'E\'': 1}
    eco_scores = {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1}
    
    # Score de base (ELECTRE TRI pessimiste)
    score = scores_electre.get(row['Categorie_Pessimiste'], 3)
    
    # Bonus/Malus Eco-Score (pondération 30%)
    if pd.notna(row.get('ecoscore_grade')) and row.get('ecoscore_grade') in eco_scores:
        score += (eco_scores[row['ecoscore_grade']] - 3) * 0.3
    
    # Bonus Bio (0.5 point)
    if row.get('bio', 0) == 1:
        score += 0.5
    
    
    # Conversion en catégorie finale
    if score >= 4.5:
        return "A''", 'Excellent choix nutritionnel et environnemental'
    elif score >= 3.5:
        return "B''", 'Bon choix avec quelques réserves'
    elif score >= 2.5:
        return "C''", 'Choix acceptable, amélioration possible'
    elif score >= 1.5:
        return "D''", 'À consommer avec modération'
    else:
        return "E''", 'À limiter dans votre alimentation'

# Application du SuperNutri-Score
df_electre_06[['SuperNutriScore', 'SuperNutriScore_Explication']] = df_electre_06.apply(
    calculer_supernutriscore, axis=1, result_type='expand'
)

print("=" * 80)
print("SUPERNUTRI-SCORE - Modèle Combiné")
print("=" * 80)
print("\nLe SuperNutri-Score combine:")
print("  ✓ Classification ELECTRE TRI (pessimiste)")
print("  ✓ Éco-Score / Green-Score")
print("  ✓ Label Bio")
print()

# Afficher les résultats
print("Exemples de SuperNutri-Score:")
colonnes_super = ['nom', 'nutriscore_grade', 'Categorie_Pessimiste', 
                 'ecoscore_grade', 'bio', 'SuperNutriScore']
df_electre_06[colonnes_super].head(15)

# %%
# Distribution du SuperNutri-Score
fig, ax = plt.subplots(figsize=(10, 6))

counts_super = df_electre_06['SuperNutriScore'].value_counts().reindex(
    ["A''", "B''", "C''", "D''", "E''"], fill_value=0
)

colors = ['#038141', '#85bb2f', '#fecb02', '#ee8100', '#e63e11']
counts_super.plot(kind='bar', ax=ax, color=colors, edgecolor='black')

ax.set_title('Distribution du SuperNutri-Score\n(Combinaison ELECTRE TRI + Eco-Score + Bio)', 
            fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Catégorie', fontsize=12, fontweight='bold')
ax.set_ylabel('Nombre de produits', fontsize=12, fontweight='bold')
ax.tick_params(axis='x', rotation=0)
ax.grid(axis='y', alpha=0.3)

# Ajouter les valeurs sur les barres
for i, v in enumerate(counts_super):
    ax.text(i, v + 0.5, str(v), ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## Comparaison Nutri-Score vs SuperNutri-Score

# %%
# Tableau comparatif
print("=" * 80)
print("COMPARAISON: Nutri-Score vs SuperNutri-Score")
print("=" * 80)

# Créer un tableau croisé
comparison_table = pd.crosstab(
    df_electre_06['nutriscore_grade'], 
    df_electre_06['SuperNutriScore'],
    margins=True,
    margins_name='Total'
)

print("\nTableau croisé:")
print(comparison_table)

# %%
# Heatmap de comparaison
fig, ax = plt.subplots(figsize=(10, 8))

# Préparer les données (sans les totaux)
comparison_data = pd.crosstab(
    df_electre_06['nutriscore_grade'], 
    df_electre_06['SuperNutriScore']
).reindex(index=['A', 'B', 'C', 'D', 'E'], columns=["A''", "B''", "C''", "D''", "E''"], fill_value=0)

sns.heatmap(comparison_data, annot=True, fmt='d', cmap='YlOrRd', 
           ax=ax, cbar_kws={'label': 'Nombre de produits'})

ax.set_title('Comparaison: Nutri-Score (OpenFoodFacts) vs SuperNutri-Score', 
            fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('SuperNutri-Score', fontsize=12, fontweight='bold')
ax.set_ylabel('Nutri-Score (OpenFoodFacts)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## Sauvegarde des Résultats
# %% [markdown]
# ## Comparaison ELECTRE TRI Pessimiste vs SuperNutri-Score

# %%
print("=" * 80)
print("COMPARAISON: ELECTRE TRI Pessimiste (λ=0.6) vs SuperNutri-Score")
print("=" * 80)

# Créer une fonction de conversion pour matcher les catégories
def convertir_categorie_electre(cat):
    """Convertir A' -> A pour la comparaison"""
    return cat.replace("'", "") if pd.notna(cat) else None

def convertir_categorie_super(cat):
    """Convertir S-A -> A pour la comparaison"""
    return cat.replace("''", "") if pd.notna(cat) else None

# Créer des colonnes temporaires pour la comparaison
df_electre_06['Electre_Simple'] = df_electre_06['Categorie_Pessimiste'].apply(convertir_categorie_electre)
df_electre_06['Super_Simple'] = df_electre_06['SuperNutriScore'].apply(convertir_categorie_super)

# Tableau croisé
comparison_electre_super = pd.crosstab(
    df_electre_06['Electre_Simple'], 
    df_electre_06['Super_Simple'],
    margins=True,
    margins_name='Total'
)

print("\nTableau croisé ELECTRE TRI Pessimiste vs SuperNutri-Score:")
print(comparison_electre_super)

# %%
# Matrice de confusion ELECTRE TRI vs SuperNutri-Score
fig, ax = plt.subplots(figsize=(10, 8))

# Préparer les données pour la heatmap
comparison_heatmap = pd.crosstab(
    df_electre_06['Categorie_Pessimiste'], 
    df_electre_06['SuperNutriScore']
).reindex(
    index=['A\'', 'B\'', 'C\'', 'D\'', 'E\''], 
    columns=["A''", "B''", "C''", "D''", "E''"], 
    fill_value=0
)

sns.heatmap(comparison_heatmap, annot=True, fmt='d', cmap='RdYlGn_r', 
           ax=ax, cbar_kws={'label': 'Nombre de produits'})

ax.set_title('Matrice de Confusion: ELECTRE TRI Pessimiste vs SuperNutri-Score\n(λ = 0.6)', 
            fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('SuperNutri-Score', fontsize=12, fontweight='bold')
ax.set_ylabel('ELECTRE TRI Pessimiste', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# %%
# Analyse de concordance entre ELECTRE TRI et SuperNutri-Score
df_clean = df_electre_06.dropna(subset=['Electre_Simple', 'Super_Simple'])

concordance_exacte = (df_clean['Electre_Simple'] == df_clean['Super_Simple']).sum()
total = len(df_clean)
taux_concordance = concordance_exacte / total if total > 0 else 0

categories_ordre = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

diff = df_clean.apply(
    lambda row: abs(categories_ordre.get(row['Electre_Simple'], 0) - 
                  categories_ordre.get(row['Super_Simple'], 0)), axis=1
)

concordance_1_cat = (diff <= 1).sum()
taux_concordance_1 = concordance_1_cat / total if total > 0 else 0

print("\n" + "=" * 80)
print("ANALYSE DE CONCORDANCE")
print("=" * 80)
print(f"Total de produits: {total}")
print(f"Concordance exacte: {concordance_exacte} produits ({taux_concordance:.1%})")
print(f"Concordance à ±1 catégorie: {concordance_1_cat} produits ({taux_concordance_1:.1%})")
print(f"\nRépartition des différences:")
for d, count in sorted(diff.value_counts().items()):
    print(f"  Différence de {d} catégorie(s): {count} produits")

# %%
# Distribution comparée
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ELECTRE TRI Pessimiste
counts_electre = df_electre_06['Categorie_Pessimiste'].value_counts().reindex(
    ['A\'', 'B\'', 'C\'', 'D\'', 'E\''], fill_value=0
)
counts_electre.plot(kind='bar', ax=axes[0], color='#3498db', edgecolor='black')
axes[0].set_title('ELECTRE TRI Pessimiste (λ = 0.6)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Catégorie', fontweight='bold')
axes[0].set_ylabel('Nombre de produits', fontweight='bold')
axes[0].tick_params(axis='x', rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# SuperNutri-Score
counts_super = df_electre_06['SuperNutriScore'].value_counts().reindex(
    ["A''", "B''", "C''", "D''", "E''"], fill_value=0
)
colors_super = ['#038141', '#85bb2f', '#fecb02', '#ee8100', '#e63e11']
counts_super.plot(kind='bar', ax=axes[1], color=colors_super, edgecolor='black')
axes[1].set_title('SuperNutri-Score', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Catégorie', fontweight='bold')
axes[1].set_ylabel('Nombre de produits', fontweight='bold')
axes[1].tick_params(axis='x', rotation=0)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()
# %%
# Sauvegarder les résultats dans des fichiers Excel
print("=" * 80)
print("SAUVEGARDE DES RÉSULTATS")
print("=" * 80)

# Fichier avec λ = 0.6
fichier_06 = 'resultats_complets_lambda_0_6.xlsx'
df_electre_06.to_excel(fichier_06, index=False)
print(f"Résultats (λ=0.6) sauvegardés: {fichier_06}")

# Fichier avec λ = 0.7
fichier_07 = 'resultats_complets_lambda_0_7.xlsx'
df_electre_07.to_excel(fichier_07, index=False)
print(f"Résultats (λ=0.7) sauvegardés: {fichier_07}")

# Sauvegarder aussi les graphiques
print("\nSauvegarde des visualisations...")
# (Les graphiques ont été affichés avec plt.show(), vous pouvez les sauvegarder manuellement)

# %% [markdown]
# ## Diagnostic des Différences avec OpenFoodFacts
# 
# Cette section analyse POURQUOI notre calcul diffère des scores OpenFoodFacts

# %%
# Importer le module de diagnostic
from sklearn.metrics import confusion_matrix

class DiagnosticNutriScore:
    """Diagnostique les différences avec OpenFoodFacts"""
    
    @staticmethod
    def analyser_differences(df: pd.DataFrame):
        df_analyse = df[df['nutriscore_grade'] != df['nutriscore_calcule']].copy()
        
        print("=" * 80)
        print("ANALYSE DÉTAILLÉE DES DIFFÉRENCES")
        print("=" * 80)
        print(f"\nNombre de produits différents: {len(df_analyse)}")
        print(f"Pourcentage: {len(df_analyse)/len(df)*100:.1f}%\n")
        
        categories_ordre = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
        df_analyse['diff_numerique'] = df_analyse.apply(
            lambda row: categories_ordre.get(row['nutriscore_calcule'], 0) - 
                       categories_ordre.get(row['nutriscore_grade'], 0), axis=1
        )
        
        print("Répartition des différences:")
        print(df_analyse['diff_numerique'].value_counts().sort_index())
        
        return df_analyse
    
    @staticmethod
    def examiner_produits_problematiques(df: pd.DataFrame, n: int = 5):
        df_diff = df[df['nutriscore_grade'] != df['nutriscore_calcule']].copy()
        
        categories_ordre = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
        df_diff['diff_abs'] = df_diff.apply(
            lambda row: abs(categories_ordre.get(row['nutriscore_grade'], 0) - 
                          categories_ordre.get(row['nutriscore_calcule'], 0)), axis=1
        )
        
        print("\n" + "=" * 80)
        print(f"EXAMEN DÉTAILLÉ DES {n} PRODUITS AVEC LES PLUS GRANDES DIFFÉRENCES")
        print("=" * 80)
        
        top_diff = df_diff.nlargest(n, 'diff_abs')
        
        for idx, row in top_diff.iterrows():
            print(f"\n{'─' * 80}")
            print(f"Produit: {row['nom']}")
            print(f"{'─' * 80}")
            print(f"Score OpenFoodFacts: {row.get('nutriscore_score', 'N/A')} → Catégorie {row['nutriscore_grade']}")
            print(f"Score Calculé: {row['score_calcule']} → Catégorie {row['nutriscore_calcule']}")
            print(f"Différence: {row['diff_abs']} catégories")
            print(f"\nComposantes:")
            print(f"   • Composante Négative: {row['composante_neg']}")
            print(f"   • Composante Positive: {row['composante_pos']}")
            print(f"\nValeurs nutritionnelles (pour 100g):")
            print(f"   • Énergie: {row['energie_kj']:.0f} kJ")
            print(f"   • Sucres: {row['sucres']:.1f} g")
            print(f"   • AG saturés: {row['graisses_saturees']:.1f} g")
            print(f"   • Sodium: {row['sodium']:.0f} mg")
            print(f"   • Protéines: {row['proteines']:.1f} g")
            print(f"   • Fibres: {row['fibres']:.1f} g")
            print(f"   • Fruits/Légumes: {row['fruits_legumes_noix']:.0f} %")

# %%
# Lancer le diagnostic
diagnostic = DiagnosticNutriScore()
df_differences = diagnostic.analyser_differences(df_resultats)

# %%
# Examiner les produits problématiques
diagnostic.examiner_produits_problematiques(df_resultats, n=10)

# %% [markdown]
# ## Explications des Différences

# %%
print("=" * 80)

print("\n" + "=" * 80)
print("INTERPRÉTATION (Nutri-Score vs OpenFoodFacts)")
print("=" * 80)

print(f"""
Sur {analyse['total_produits']} produits comparables :
- concordance exacte : {analyse['taux_concordance_exacte']:.1%}
- concordance à ±1 catégorie : {analyse['taux_concordance_1_categorie']:.1%}

Les écarts viennent souvent de données d'entrée incertaines (ex : % fruits/légumes),
des effets de seuil, ou d'arrondis/conversions.
""")


# %% [markdown]
# ## Pourquoi ELECTRE TRI Diffère 

# %%
print("=" * 80)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 80)

# Statistiques sur les composantes du Nutri-Score
print("\nComposantes Nutri-Score:")
print(df_resultats[['composante_neg', 'composante_pos', 'score_calcule']].describe())

# %%
# Corrélation entre score calculé et score OpenFoodFacts
if 'nutriscore_score' in df_resultats.columns:
    print("\nCorrélation entre score calculé et score OpenFoodFacts:")
    
    df_corr = df_resultats[['nutriscore_score', 'score_calcule']].dropna()
    correlation = df_corr.corr().iloc[0, 1]
    print(f"Coefficient de corrélation de Pearson: {correlation:.3f}")
    
    # Scatter plot
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(df_corr['nutriscore_score'], df_corr['score_calcule'], 
              alpha=0.6, s=50, c='steelblue', edgecolors='black')
    
    # Ligne de régression
    z = np.polyfit(df_corr['nutriscore_score'], df_corr['score_calcule'], 1)
    p = np.poly1d(z)
    ax.plot(df_corr['nutriscore_score'], p(df_corr['nutriscore_score']), 
           "r--", linewidth=2, label=f'Régression linéaire')
    
    # Ligne y=x (concordance parfaite)
    min_val = min(df_corr['nutriscore_score'].min(), df_corr['score_calcule'].min())
    max_val = max(df_corr['nutriscore_score'].max(), df_corr['score_calcule'].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'g--', 
           linewidth=2, alpha=0.7, label='Concordance parfaite')
    
    ax.set_xlabel('Score OpenFoodFacts', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score Calculé', fontsize=12, fontweight='bold')
    ax.set_title(f'Corrélation des Scores (r = {correlation:.3f})', 
                fontsize=14, fontweight='bold', pad=20)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ## Produits avec les Plus Grandes Différences

# %%
# Identifier les produits avec les plus grandes différences
categories_ordre = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

df_diff = df_resultats.dropna(subset=['nutriscore_grade', 'nutriscore_calcule']).copy()
df_diff['diff_abs'] = df_diff.apply(
    lambda row: abs(categories_ordre.get(row['nutriscore_grade'], 0) - 
                  categories_ordre.get(row['nutriscore_calcule'], 0)), axis=1
)

print("=" * 80)
print("TOP 10 PRODUITS AVEC LES PLUS GRANDES DIFFÉRENCES")
print("=" * 80)

top_differences = df_diff.nlargest(10, 'diff_abs')
colonnes_diff = ['nom', 'nutriscore_grade', 'nutriscore_calcule', 
                'score_calcule', 'composante_neg', 'composante_pos', 'diff_abs']
print(top_differences[colonnes_diff].to_string(index=False))
# Produits dont le Nutri-Score calculé est différent de celui d'OpenFoodFacts
df_mismatch = df_diff[df_diff['diff_abs'] > 0].copy()

# On peut garder les mêmes colonnes que pour l’affichage, plus si tu veux
colonnes_export = [
    'nom',
    'nutriscore_grade',      # Nutri-Score OFF (réel)
    'nutriscore_calcule',    # Nutri-Score calculé
    'score_calcule',
    'composante_neg',
    'composante_pos',
    'diff_abs'
]

# Ne garder que les colonnes qui existent vraiment (au cas où)
colonnes_export = [c for c in colonnes_export if c in df_mismatch.columns]

fichier_mismatch = "produits_nutriscore_diff_OFF_vs_calcule.xlsx"
df_mismatch[colonnes_export].to_excel(fichier_mismatch, index=False)

print(f"\nFichier créé avec tous les produits dont le Nutri-Score diffère d'OFF : {fichier_mismatch}")
print(f"Nombre de produits concernés : {len(df_mismatch)}")


# %% [markdown]
# ---
# ---

# %%
print("=" * 80)
print("ANALYSE TERMINÉE")
print("=" * 80)
print("=" * 80)
print("TOP 10 PRODUITS AVEC LES PLUS GRANDES DIFFÉRENCES")
print("=" * 80)

top_differences = df_diff.nlargest(10, 'diff_abs')
colonnes_diff = ['nom', 'nutriscore_grade', 'nutriscore_calcule', 
                'score_calcule', 'composante_neg', 'composante_pos', 'diff_abs']
print(top_differences[colonnes_diff].to_string(index=False))

# %% [markdown]
# ---
# ---

# %%
print("=" * 80)
print("ANALYSE TERMINÉE")
print("=" * 80)

print(f"""
Statistiques Globales:
   • Total de produits analysés: {len(df)}
   • Nutri-Score calculé: {df_resultats['nutriscore_calcule'].notna().sum()} produits
   
Nutri-Score vs OpenFoodFacts:
   • Taux de concordance exacte: {analyse['taux_concordance_exacte']:.1%}
   • Taux de concordance ±1 catégorie: {analyse['taux_concordance_1_categorie']:.1%}
   
Classification ELECTRE TRI:
   • Méthode pessimiste appliquée avec λ=0.6 et λ=0.7
   • Méthode optimiste appliquée avec λ=0.6 et λ=0.7
   
SuperNutri-Score:
   • Modèle combiné créé avec succès
   • Intègre: ELECTRE TRI + Eco-Score + Label Bio
   
Fichiers générés:
   • resultats_complets_lambda_0_6.xlsx
   • resultats_complets_lambda_0_7.xlsx
   
Visualisations créées:
   • Matrice de confusion
   • Distribution des catégories
   • Comparaisons ELECTRE TRI
   • SuperNutri-Score distribution
   • Corrélation des scores
""")